In [1]:
import pandas as pd
import numpy as np

# web scraper

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import lxml.html
import re
import time

# write to csv file

import csv

# joblib is a set of tools to provide lightweight pipelining in Python. It provides utilities for saving and loading Python objects that make use of NumPy data structures, efficiently.
import joblib

In [87]:
url = 'https://outfithustler.com/collections/kids-fashion'
#url='https://www.currys.ie/ieen/search-keywords/xx_xx_xx_xx_xx/-wk22_headphones_ie-/xx-criteria.html'

# to avoid opening browser while using selenium
option = webdriver.ChromeOptions()
option.add_argument('headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),options=option)

driver.get(url)
time.sleep(1)

# get source code -- type: str
html_source = driver.page_source

# key
html = lxml.html.fromstring(html_source)

# obtain all the text under the 'div' tags
items = html.xpath("//div//text()")

pattern = re.compile("^\s+|\s+$|\n")

clause_text = ""

for item in items:
    line = re.sub(pattern, "", item)
    if len(item) > 1:
        clause_text += line +"\n"

driver.quit()



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/zenglan/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


In [88]:
raw_text = clause_text

# the beginning character of the content, which is the sign we should ignore the content
ignore_str = ',.;{}?#/'

# the content we are going to keep to send to models.
content_list = []

# only keep the content that has words count from 2 to 50 (includes).
for line in raw_text.split('\n'):
    if 1<len(line.split())<=45 and line[0] not in ignore_str:
        content_list.append([line])

In [89]:
header = ['content']

# create a csv file to save the filtered content for later model analysis.
with open('outfit-kids.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write the data
    writer.writerows(content_list)

------------------
# Check Presence

In [101]:
# Loading the saved model with joblib
presence_model = joblib.load('svm_presence_classifier.joblib')
presence_cv = joblib.load('presence_CountVectorizer.joblib')

# New dataset to predict
presence_pred = pd.read_csv('outfit-kids.csv')

In [95]:
presence_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  196 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


In [102]:
# apply the pretrained model to the new content data
pre_pred_vec = presence_model.predict(presence_cv.transform(presence_pred['content']))

presence_pred['presence'] = pre_pred_vec.tolist()

# dark pattern content are those where the predicted result equals to 0.
dark = presence_pred.loc[presence_pred['presence']==0]

dark.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 0 to 191
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   content   29 non-null     object
 1   presence  29 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 696.0+ bytes


In [103]:
dark.to_csv('TestResult/outfit-kids-svm.csv', index = False, header = True)